# Рекомендация тарифов

В моём распоряжении данные о поведении клиентов, которые уже перешли на тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф.

Необходимо построить модель с максимально большим значением *accuracy* и довести долю правильных ответов по крайней мере до 0.75.

## Откроем и изучим файл

In [1]:
import pandas as pd

df = pd.read_csv('users_behavior.csv')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


<b> Описание данных </b>
* сalls — количество звонков,
* minutes — суммарная длительность звонков в минутах,
* messages — количество sms-сообщений,
* mb_used — израсходованный интернет-трафик в Мб,
* is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## Разобьем данные на выборки

In [2]:
from sklearn.model_selection import train_test_split

features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

features_train, features_test, target_train, target_test = train_test_split(features,
                                                                            target,
                                                                            test_size=0.4,
                                                                            random_state=12345)

features_valid, features_test, target_valid, target_test = train_test_split(features_test,
                                                                            target_test,
                                                                            test_size=0.5,
                                                                            random_state=12345)

## Исследуем модели

### Дерево решений

In [3]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

tree_best_model = None
tree_best_result = 0
for depth in range(1, 6):
    model = DecisionTreeClassifier(random_state=12345, max_depth= depth)
    model.fit(features_train, target_train) 
    predictions_valid = model.predict(features_valid) 
    result = accuracy_score(target_valid, predictions_valid)
    if result > tree_best_result:
        tree_best_model = model
        tree_best_result = result

print("Accuracy лучшей модели:", tree_best_result)
print("Глубина дерева:", tree_best_model.max_depth)

Accuracy лучшей модели: 0.7853810264385692
Глубина дерева: 3


### Случайный лес

In [4]:
from sklearn.ensemble import RandomForestClassifier

forest_best_model = None
forest_best_result = 0
for est in range(1, 25):
    model = RandomForestClassifier(random_state=12345, n_estimators=est)
    model.fit(features_train, target_train)
    result = model.score(features_valid, target_valid)
    if result > forest_best_result:
        forest_best_model = model
        forest_best_result = result

print("Accuracy лучшей модели:", forest_best_result)
print("Количество деревьев в лесу:", forest_best_model.n_estimators)

Accuracy лучшей модели: 0.7947122861586314
Количество деревьев в лесу: 23


Настроим гиперпараметр criterion с gini (по умолчанию) на entropy.

In [5]:
model = RandomForestClassifier(random_state=12345, n_estimators=23, criterion='entropy')
model.fit(features_train, target_train)
result = model.score(features_valid, target_valid)
print("Accuracy модели:", result)

Accuracy модели: 0.7838258164852255


Гиперпараметр gini даёт большую точность. Изменим минимальное количество объектов в листе (min_samples_split) на 2 (по умолчанию 1)

In [6]:
model = RandomForestClassifier(random_state=12345, n_estimators=23, min_samples_leaf=2)
model.fit(features_train, target_train)
result = model.score(features_valid, target_valid)
print("Accuracy модели:", result)

Accuracy модели: 0.8009331259720062


Точность стала выше. Попробуем изменить минимальное количество примеров для разделения (min_samples_split).

In [7]:
model = RandomForestClassifier(random_state=12345, n_estimators=23, min_samples_leaf=2, min_samples_split=4)
model.fit(features_train, target_train)
result = model.score(features_valid, target_valid)
print("Accuracy модели:", result)

Accuracy модели: 0.8009331259720062


Небольшое измение гиперпараметра ничего не меняет, если его увеличить в разы, то точность лишь снижается.

### Логистическая регрессия

In [8]:
from sklearn.linear_model import LogisticRegression 

model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000) 
model.fit(features_train, target_train) 
result = model.score(features_valid, target_valid) 
print("Accuracy модели:", result)

Accuracy модели: 0.7107309486780715


Наибольшая доля правильных ответов (0.8) достигается при использовании модели случайного леса со следующими гиперпараметрами:

In [9]:
forest_best_model = RandomForestClassifier(random_state=12345, n_estimators=23, min_samples_leaf=2)
forest_best_model.fit(features_train, target_train)
forest_best_model

RandomForestClassifier(min_samples_leaf=2, n_estimators=23, random_state=12345)

## Проверим модель на тестовой выборке

In [10]:
test_predictions = forest_best_model.predict(features_test)
accuracy = accuracy_score(target_test, test_predictions)
print('Точность модели слуйчаного леса на тестовой выборке', accuracy)

Точность модели слуйчаного леса на тестовой выборке 0.7916018662519441


<b> Вывод </b>
В ходе исследования данные были загружены, изучены и разделены на обучающую, валидационную и тестовую выборки. Рассмотрено 3 модели: 
- дерево решений;
- случайный лес;
- логистическая регрессия.

Наибольшее Accuracy (0.8) в данном случае дает случайный лес с гипперпараметрами min_samples_leaf=2, n_estimators=23, random_state=12345.